In [15]:
#!wget https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt

In [16]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding
from tensorflow.keras.optimizers import Adam

In [17]:
# Path to the shakespeare.txt file
file_path = 'shakespeare.txt'

# Open and read the file
with open(file_path, 'r', encoding='utf-8') as file:
    shakespeare_text = file.read()

# Print the first 500 characters (optional, just to preview the content)
print(shakespeare_text[:500])  # Adjust as needed to read more or less


First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [18]:
text = shakespeare_text
chars = sorted(set(text))  # All unique characters
char_to_idx = {char: idx for idx, char in enumerate(chars)}  # Mapping from char to index
idx_to_char = {idx: char for idx, char in enumerate(chars)}

In [19]:
print(char_to_idx)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}


In [20]:
seq_length = 10  # Number of characters in each sequence
vocab_size = len(chars)  # Size of vocabulary
embedding_dim = 64  # Embedding dimensions
rnn_units = 128  # Number of RNN units
batch_size = 32

In [21]:
def create_sequences(text, seq_length):
    X = []
    y = []
    for i in range(len(text) - seq_length):
        X.append([char_to_idx[char] for char in text[i:i + seq_length]])
        y.append(char_to_idx[text[i + seq_length]])
    return np.array(X), np.array(y)

In [22]:

X, y = create_sequences(text, seq_length)
print(X)
print(y)

[[18 47 56 ... 47 58 47]
 [47 56 57 ... 58 47 64]
 [56 57 58 ... 47 64 43]
 ...
 [ 1 39 56 ... 49 47 52]
 [39 56 58 ... 47 52 45]
 [56 58  1 ... 52 45  8]]
[64 43 52 ... 45  8  0]


In [23]:
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim),
    SimpleRNN(rnn_units, activation='relu'),  # ReLU activation function
    Dense(vocab_size, activation='softmax')  # Softmax for output layer
])

In [24]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy')

In [25]:
# use this if using tensorflow/keras 2.x

# checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#     filepath='model_weights_epoch_{epoch:02d}.h5',  # Save file with epoch number in the name
#     save_weights_only=True,                         # Only save the weights, not the full model
#     save_freq='epoch',                              # Save at the end of each epoch
#     period=2,                                       # Save every 2 epochs
#     verbose=1                                       # Print saving info to the console
# )

In [28]:
def save_weights_every_2_epochs(epoch, logs):
    if (epoch + 1) % 2 == 0:  # Save every 2 epochs
        model.save_weights(f'model_weights_epoch_{epoch+1}.weights.h5')
        print(f'\nSaved weights at epoch {epoch+1}')

checkpoint_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=save_weights_every_2_epochs)

In [29]:
model.fit(X, y, batch_size=batch_size, epochs=100, callbacks=[checkpoint_callback])

Epoch 1/100
  102/34856 ━━━━━━━━━━━━━━━━━━━━ 1:09 2ms/step - loss: 1.6011

34856/34856 ━━━━━━━━━━━━━━━━━━━━ 79s 2ms/step - loss: 1.6097
Epoch 2/100
34847/34856 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.5936
Saved weights at epoch 2
34856/34856 ━━━━━━━━━━━━━━━━━━━━ 87s 3ms/step - loss: 1.5936
Epoch 3/100
34856/34856 ━━━━━━━━━━━━━━━━━━━━ 77s 2ms/step - loss: 1.5798
Epoch 4/100
34837/34856 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.5754
Saved weights at epoch 4
34856/34856 ━━━━━━━━━━━━━━━━━━━━ 87s 2ms/step - loss: 1.5754
Epoch 5/100
34856/34856 ━━━━━━━━━━━━━━━━━━━━ 88s 3ms/step - loss: 1.5703
Epoch 6/100
34856/34856 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.5655
Saved weights at epoch 6
34856/34856 ━━━━━━━━━━━━━━━━━━━━ 89s 3ms/step - loss: 1.5655
Epoch 7/100
34856/34856 ━━━━━━━━━━━━━━━━━━━━ 88s 3ms/step - loss: 1.5646
Epoch 8/100
34850/34856 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.5633
Saved weights at epoch 8
34856/34856 ━━━━━━━━━━━━━━━━━━━━ 84s 2ms/step - loss: 1.5633
Epoch 9/100
34856/34856 ━━━━━━━━━━━━━━━━━━━━ 72s 2ms/step - loss: 1.5601
Epoch 10/100
34

In [ ]:
model.save('my_rnn_model')


In [ ]:
def generate_text(model, start_string, gen_length=100, temperature=1.0):
    # Convert start_string to numbers (vectorize)
    input_eval = [char_to_idx[char] for char in start_string]
    input_eval = np.expand_dims(input_eval, 0)  # Add batch dimension

    generated_text = []

    for i in range(gen_length):
        # Get predictions for the current input sequence
        predictions = model(input_eval)

        # Remove the batch dimension from predictions
        predictions = tf.squeeze(predictions, 0)

        # Apply temperature scaling to logits
        predictions = predictions / temperature

        # Now predictions are a 1D tensor, so expand the dimension to make it 2D
        predictions = tf.expand_dims(predictions, 0)

        # Sample the next character from the probability distribution
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Add the predicted character to the generated text
        generated_text.append(idx_to_char[predicted_id])

        # Update the input to include the predicted character
        input_eval = np.append(input_eval[:, 1:], np.expand_dims([predicted_id], 0), axis=1)

    return start_string + ''.join(generated_text)


In [ ]:

print(generate_text(model, start_string="hello"))
